In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
%matplotlib inline

This notebook assumes that the ETHEC dataset (zip file) was downloaded and placed in the same folder as the code. The dataset can be downloaded from: https://www.research-collection.ethz.ch/handle/20.500.11850/365379

In [ ]:
path = os.getcwd()

# Check if running in Colab
try:
  from google.colab import drive
  IN_COLAB=True
  print("Running in Colab")
  # Mount Google Drive
  drive.mount('/content/drive', force_remount=True)
  # Change directory
  %cd "YOUR_PATH"
except:
  IN_COLAB=False
  print("Running locally")

In [ ]:
from pairs import distinct_pairs_func, make_pairs, contrastive_loss
from algorithm import *
from synthetic_data import *

### Load data

In [ ]:
! unzip "ETHEC_v02"

In [ ]:
# get meta data from files

def extract_details(path):
  df = pd.read_json(path)
  df = df.transpose()
  df = df[['image_path', 'image_name', 'family', 'subfamily', 'genus', 'specific_epithet']]
  return df

train_df = extract_details('ETHEC_dataset/splits/train.json')
test_df = extract_details('ETHEC_dataset/splits/test.json')
val_df = extract_details('ETHEC_dataset/splits/val.json')

In [ ]:
# focus on Lycaenidae and Nymphalidae families
chosen_families = ['Lycaenidae', 'Nymphalidae']
minority = 'Lycaenidae'

# subset chosen families
train_df = train_df[train_df['family'].isin(chosen_families)]
test_df = test_df[test_df['family'].isin(chosen_families)]
val_df = val_df[val_df['family'].isin(chosen_families)]
df = pd.concat([train_df, test_df, val_df], ignore_index=True, axis=0)

minority_classes = (df[df['family']==minority]['specific_epithet']).unique()
majority_classes = (df[df['family']!=minority]['specific_epithet']).unique()

In [ ]:
C = df['specific_epithet']
Nc = len(minority_classes) + len(majority_classes)

p_minor = 0.1 # proportion of minority group
test_p = 0.2 # proportion of classes to save for test (under distribution shifyt)

N_minor_test = int(len(minority_classes) * test_p)
N_major_test = int(N_minor_test * (p_minor)/(1-p_minor))

In [ ]:
# select test classes (distribution shift scenario)

test_minority_classes = np.random.choice(minority_classes, N_minor_test, replace=False)
test_majority_classes = np.random.choice(majority_classes, N_major_test, replace=False)
unq_c_test = np.hstack([test_minority_classes, test_majority_classes])

In [ ]:
assert np.mean(np.isin(test_minority_classes, minority_classes))==1

In [ ]:
print("minority proportion in training: {:.4f}, majority proportion in training: {:.4f}".format(np.mean(np.isin(unq_c_test, test_minority_classes)), np.mean(np.isin(unq_c_test, test_majority_classes))))

In [ ]:
remaining_minorty_classes = np.array([c for c in minority_classes if c not in test_minority_classes])
remaining_majority_classes = np.array([c for c in majority_classes if c not in test_majority_classes])

In [ ]:
# select train classes

N_minor_train = min(int(len(remaining_majority_classes) * (p_minor)/(1-p_minor)), len(remaining_minorty_classes))
N_major_train = min(len(remaining_majority_classes), int(N_minor_train * (1 - p_minor)/p_minor))

train_minority_classes = np.random.choice(minority_classes, N_minor_train, replace=False)
train_majority_classes = np.random.choice(majority_classes, N_major_train, replace=False)
unq_c_train = np.hstack([train_minority_classes, train_majority_classes])

In [ ]:
assert np.mean(np.isin(train_minority_classes, minority_classes))==1

In [ ]:
print("minority proportion in training: {:.4f}, majority proportion in training: {:.4f}".format(np.mean(np.isin(unq_c_train, train_minority_classes)), np.mean(np.isin(unq_c_train, train_majority_classes))))

In [ ]:
# set apart validation data (in-distrinution setting)

unq_c_val = np.random.choice(unq_c_train, int(0.4*len(unq_c_train)), replace=False)
unq_c_train = np.array([c for c in unq_c_train if c not in unq_c_val])
Nc = len(unq_c_val) + len(unq_c_train) + len(unq_c_test)

In [ ]:
def get_file_names(df, c_list, r = 10):
  all_files = []
  for c in c_list:
    paths = list(df.image_path[df['specific_epithet'].isin(c_list)])
    names = list(df.image_name[df['specific_epithet'].isin(c_list)])
    files = [os.path.join('ETHEC_dataset', 'IMAGO_build_test_resized', paths[i], names[i]) for i in range(len(paths))]
    all_files.append(np.random.choice(files, r, replace=False))
  return files

train_files, val_files, test_files = get_file_names(df, unq_c_train), get_file_names(df, unq_c_val), get_file_names(df, unq_c_test)

In [ ]:
# extract images

SIZE = (100,100)

def load_img(path, size):
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, size)
  return np.array(img)

# data
z_train = np.array([load_img(file, SIZE) for file in train_files]) / 255.0
z_test = np.array([load_img(file, SIZE) for file in test_files]) / 255.0
z_val = np.array([load_img(file, SIZE) for file in val_files]) / 255.0

In [ ]:
# class names
c_train_name = list(df.specific_epithet[df['specific_epithet'].isin(unq_c_train)])
c_test_name = list(df.specific_epithet[df['specific_epithet'].isin(unq_c_test)])
c_val_name = list(df.specific_epithet[df['specific_epithet'].isin(unq_c_val)])

# attributes
A_train_name = [list(df.family[df.specific_epithet==c])[0] for c in c_train_name]
A_test_name = [list(df.family[df.specific_epithet==c])[0] for c in c_test_name]
A_val_name = [list(df.family[df.specific_epithet==c])[0] for c in c_val_name]

attribute_dict = {'Nymphalidae': 0, 'Lycaenidae':1}
A_train = np.array([attribute_dict[name] for name in A_train_name])
A_test = np.array([attribute_dict[name] for name in A_test_name])
A_val = np.array([attribute_dict[name] for name in A_val_name])

# classes
unq_c = np.unique(np.hstack([c_train_name, c_test_name, c_val_name]))
class_dict = {}
for i, c in enumerate(unq_c):
  class_dict[c] = i

c_train, c_test, c_val = np.array([class_dict[c] for c in c_train_name]), np.array([class_dict[c] for c in c_test_name]), np.array([class_dict[c] for c in c_val_name])
unq_c_train, unq_c_test, unq_c_val = np.unique(c_train), np.unique(c_test), np.unique(c_val)

In [ ]:
# keep species with at least 5 images

def filter_images(z, c_list, selected_c, names_list, A, min_images):
  c_idx = np.array([np.isin(c_list, c) for c in selected_c])
  selected_c = selected_c[np.sum(c_idx, axis=1) >= min_images]
  idx = np.array([np.random.choice(np.where(np.isin(c_list, c))[0], min_images, replace=False) for c in selected_c]).flatten()
  return c_list[idx], z[idx], np.array(names_list)[idx], A[idx]

min_images = 5

c_train, z_train, c_train_name, A_train = filter_images(z_train, c_train, unq_c_train, c_train_name, A_train, min_images)
c_val, z_val, c_val_name, A_val = filter_images(z_val, c_val, unq_c_val, c_val_name, A_val, min_images)
c_test, z_test, c_test_name, A_test = filter_images(z_test, c_test, unq_c_test, c_test_name, A_test, min_images)

unq_c_train, unq_c_test, unq_c_val = np.unique(c_train), np.unique(c_test), np.unique(c_val)
unq_c = np.unique(np.hstack([c_train, c_test, c_val]))

In [ ]:
pos_per_class_train, pos_per_class_test = 10, 10

# generate pairs
train_z1, train_z2, train_y, train_Cs = make_pairs(z_train, c_train, pos_per_class=pos_per_class_train)
val_z1, val_z2, val_y, val_Cs = make_pairs(z_val, c_val, pos_per_class=pos_per_class_train)
test_z1, test_z2, test_y, test_Cs = make_pairs(z_test, c_test, pos_per_class_test)

### Class sampling

In [ ]:
classes_in_env = 2
classes_in_env_test = 2

n_sim_envs = 200

In [ ]:
n_envs =  10**6

In [ ]:
train_envs = []
for i in range(n_envs):
  e = np.random.choice(unq_c_train, classes_in_env, replace=False)
  train_envs.append(e)

train_envs = np.array(train_envs)

### Models

In [ ]:
s1, s2, s3 = SIZE[0], SIZE[1], 3

In [ ]:
def init_representation():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(s1, s2, s3)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(16))
    return model

Parameters

In [ ]:
lr = 1e-5
n_pairs = 10**5

ERM_factor = 0.0
CLoVE_factor = 0.05
VarAUC_factor = 0.2
VarREx_factor = 0.1

IRM_factor = 0.02
l2_regularizer_weight = tf.constant(0.01)

Initialization

In [ ]:
init_g = init_representation()

ERM_g = init_representation()
IRM_g = init_representation()
CLoVE_g = init_representation()
VarREx_g = init_representation()
VarAUC_g = init_representation()

ERM

In [ ]:
ERM_g.set_weights(init_g.get_weights())

In [ ]:
ERM_optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

In [ ]:
ERM_g, ERM_losses, ERM_Ns, ERM_test_aucs, ERM_val_aucs = training(ERM_optimizer, ERM_g, z_train, c_train, train_envs, val_z1, val_z2, val_y,
                                                                  test_z1, test_z2, test_y, n_pairs, pos_per_class_train, ERM_factor, n_sim_envs, penalty_type=None)

In [ ]:
ERM_auc_train = evaluate(ERM_g, train_z1, train_z2, train_y)
ERM_auc_val = evaluate(ERM_g, val_z1, val_z2, val_y)
ERM_auc_test = evaluate(ERM_g, test_z1, test_z2, test_y)

ERM_auc_train, ERM_auc_val, ERM_auc_test
print("Trainig data: {:.4f}, In-distribution: {:.4f}, Distribution-shift: {:.4f}".format(ERM_auc_train, ERM_auc_val, ERM_auc_test))

IRM

In [ ]:
IRM_g.set_weights(init_g.get_weights())

In [ ]:
IRM_optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

In [ ]:
IRM_g, IRM_losses, IRM_Ns, IRM_test_aucs, IRM_val_aucs = training(IRM_optimizer, IRM_g, z_train, c_train, train_envs, val_z1, val_z2, val_y,
                                                                                 test_z1, test_z2, test_y, n_pairs, pos_per_class_train, IRM_factor, n_sim_envs,
                                                                                 penalty_type='IRM')

In [ ]:
IRM_auc_train = evaluate(IRM_g, train_z1, train_z2, train_y)
IRM_auc_val = evaluate(IRM_g, val_z1, val_z2, val_y)
IRM_auc_test = evaluate(IRM_g, test_z1, test_z2, test_y)

print("Trainig data: {:.4f}, In-distribution: {:.4f}, Distribution-shift: {:.4f}".format(IRM_auc_train, IRM_auc_val, IRM_auc_test))

CLoVe

In [ ]:
CLoVE_g.set_weights(init_g.get_weights())

In [ ]:
CLoVE_optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

In [ ]:
CLoVE_g, CLoVE_losses, CLoVE_Ns, CLoVE_test_aucs, CLoVE_val_aucs = training(CLoVE_optimizer, CLoVE_g, z_train, c_train, train_envs, val_z1, val_z2, val_y,
                                                                            test_z1, test_z2, test_y, n_pairs, pos_per_class_train, CLoVE_factor, n_sim_envs,
                                                                            penalty_type='CLoVE')

In [ ]:
CLoVE_auc_train = evaluate(CLoVE_g, train_z1, train_z2, train_y)
CLoVE_auc_val = evaluate(CLoVE_g, val_z1, val_z2, val_y)
CLoVE_auc_test = evaluate(CLoVE_g, test_z1, test_z2, test_y)

print("Trainig data: {:.4f}, In-distribution: {:.4f}, Distribution-shift: {:.4f}".format(CLoVE_auc_train, CLoVE_auc_val, CLoVE_auc_test))

VarREx

In [ ]:
VarREx_g.set_weights(init_g.get_weights())

In [ ]:
VarREx_optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

In [ ]:
VarREx_g, VarREx_losses, VarREx_Ns, VarREx_test_aucs, VarREx_val_aucs = training(VarREx_optimizer, VarREx_g, z_train, c_train, train_envs, val_z1, val_z2, val_y,
                                                                                 test_z1, test_z2, test_y, n_pairs, pos_per_class_train, VarREx_factor, n_sim_envs,
                                                                                 penalty_type='VarREx')

In [ ]:
VarREx_auc_train = evaluate(VarREx_g, train_z1, train_z2, train_y)
VarREx_auc_val = evaluate(VarREx_g, val_z1, val_z2, val_y)
VarREx_auc_test = evaluate(VarREx_g, test_z1, test_z2, test_y)

print("Trainig data: {:.4f}, In-distribution: {:.4f}, Distribution-shift: {:.4f}".format(VarREx_auc_train, VarREx_auc_val, VarREx_auc_test))

VarAUC

In [ ]:
VarAUC_g.set_weights(init_g.get_weights())

In [ ]:
VarAUC_optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

In [ ]:
VarAUC_g, VarAUC_losses, VarAUC_Ns, VarAUC_test_aucs, VarAUC_val_aucs = training(VarAUC_optimizer, VarAUC_g, z_train, c_train, train_envs, val_z1, val_z2, val_y,
                                                                                 test_z1, test_z2, test_y, n_pairs, pos_per_class_train, VarAUC_factor, n_sim_envs,
                                                                                 penalty_type='VarAUC')

In [ ]:
VarAUC_auc_train = evaluate(VarAUC_g, train_z1, train_z2, train_y)
VarAUC_auc_val = evaluate(VarAUC_g, val_z1, val_z2, val_y)
VarAUC_auc_test = evaluate(VarAUC_g, test_z1, test_z2, test_y)

print("Trainig data: {:.4f}, In-distribution: {:.4f}, Distribution-shift: {:.4f}".format(VarAUC_auc_train, VarAUC_auc_val, VarAUC_auc_test))

### Comparing results

In [ ]:
plt.plot(ERM_Ns, ERM_val_aucs, '--', color='C3')
plt.plot(ERM_Ns, ERM_test_aucs,  label='ERM', color='C3')

plt.plot(IRM_Ns, IRM_val_aucs, '--', color='C1', alpha=0.8)
plt.plot(IRM_Ns, IRM_test_aucs, label='IRM ', color='C1', alpha=0.8)

plt.plot(CLoVE_Ns, CLoVE_val_aucs, '--', color='C0', alpha=0.8)
plt.plot(CLoVE_Ns, CLoVE_test_aucs, label='CLOvE', color='C0', alpha=0.8)

plt.plot(VarREx_Ns, VarREx_val_aucs, '--', color='C4', alpha=0.8)
plt.plot(VarREx_Ns, VarREx_test_aucs, label='VarREx', color='C4', alpha=0.8)

plt.plot(VarAUC_Ns, VarAUC_val_aucs, '--',  color='C2')
plt.plot(VarAUC_Ns, VarAUC_test_aucs, label='VarAUC', color='C2')


plt.xlabel('Data Points (pairs)')
plt.ylabel('AUC')
plt.legend(loc=4);

In [ ]:
# split trainin data by attribute
z_a1, c_a1 = z_train[A_train==1], c_train[A_train==1]
z_a0, c_a0 = z_train[A_train==0], c_train[A_train==0]

# make pairs
z1_a1, z2_a1, y_a1, c_a1 = make_pairs(z_a1, c_a1, pos_per_class_train)
z1_a0, z2_a0, y_a0, c_a0 = make_pairs(z_a0, c_a0, pos_per_class_train)

# ERM representations
z1_hat_a1_ERM, z2_hat_a1_ERM = ERM_g(z1_a1), ERM_g(z2_a1)
z1_hat_a0_ERM, z2_hat_a0_ERM = ERM_g(z1_a0), ERM_g(z2_a0)

# VarAUC representations
z1_hat_a1_VarAUC, z2_hat_a1_VarAUC = VarAUC_g(z1_a1), VarAUC_g(z2_a1)
z1_hat_a0_VarAUC, z2_hat_a0_VarAUC = VarAUC_g(z1_a0), VarAUC_g(z2_a0)

# unpenalized losses
def raw_loss(z1_hat, z2_hat, y_true, margin=0.5):
  dist = cosine_distance(z1_hat, z2_hat)
  l = contrastive_loss(y_true, dist, margin)
  return l, dist

# on a1
base_loss_a1_ERM, dist_a1_ERM = raw_loss(z1_hat_a1_ERM, z2_hat_a1_ERM, y_a1)
base_loss_a1_VarAUC, dist_a1_VarAUC = raw_loss(z1_hat_a1_VarAUC, z2_hat_a1_VarAUC, y_a1)
dist_a1_ERM, dist_a1_VarAUC = dist_a1_ERM.numpy(), dist_a1_VarAUC.numpy()

# on a0
base_loss_a0_ERM, dist_a0_ERM = raw_loss(z1_hat_a0_ERM, z2_hat_a0_ERM, y_a0)
base_loss_a0_VarAUC, dist_a0_VarAUC = raw_loss(z1_hat_a0_VarAUC, z2_hat_a0_VarAUC, y_a0)
dist_a0_ERM, dist_a0_VarAUC = dist_a0_ERM.numpy(), dist_a0_VarAUC.numpy()

In [ ]:
def get_bins(diff, n_bins=19):
  w = np.ptp(diff)/n_bins
  u = np.ceil((max(diff)-w/2)/w)*w + w/2
  l = np.ceil((abs(min(diff))-w/2)/w)*w + w/2
  return np.linspace(-l, u, n_bins+2)

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(8,8))

t1 = len(base_loss_a1_ERM[y_a1==0])
t0 = len(base_loss_a0_ERM[y_a0==0])

n_bins=19

diff_00 = base_loss_a1_ERM[y_a1==0] - base_loss_a1_VarAUC[y_a1==0]
bins_00 = get_bins(diff_00, n_bins)
axs[0,0].hist(diff_00, bins=bins_00, alpha=0.5, weights=(np.ones(t1)/t1), color='C5', ec='C5')
axs[0,0].axvline(0, c='k', linestyle=':')
axs[0,0].set_title('Nymphalidae (minority in training)', fontsize=11)
axs[0,0].set_ylabel(r'$y=0$', fontsize=11)
xl = max(abs(bins_00))*1.1
axs[0,0].set_xlim(-xl, xl)

diff_01 = base_loss_a0_ERM[y_a0==0] - base_loss_a0_VarAUC[y_a0==0]
bins_01 = get_bins(diff_01, n_bins)
axs[0,1].hist(diff_01, bins=bins_01, alpha=0.5, weights=(np.ones(t0)/t0), color='C5', ec='C5')
axs[0,1].axvline(0, c='k', linestyle=':')
axs[0,1].set_title('Lycaenidae (majority in training)', fontsize=11)
xl = max(abs(bins_01))*1.1
axs[0,1].set_xlim(-xl, xl)

t1 = len(base_loss_a1_ERM[y_a1==1])
t0 = len(base_loss_a0_ERM[y_a0==1])

diff_10 = base_loss_a1_ERM[y_a1==1] - base_loss_a1_VarAUC[y_a1==1]
bins_10 = get_bins(diff_10, n_bins)
axs[1,0].hist(diff_10, bins=bins_10, alpha=0.5, weights=(np.ones(t1)/t1), color='C7', ec='C7')
axs[1,0].axvline(0, c='k', linestyle=':')
axs[1,0].set_ylabel(r'$y=1$', fontsize=11)
xl = max(abs(bins_10))*1.1
axs[1,0].set_xlim(-xl, xl)

diff_11 = base_loss_a0_ERM[y_a0==1] - base_loss_a0_VarAUC[y_a0==1]
bins_11 = get_bins(diff_11, n_bins)
axs[1,1].hist(diff_11, bins=bins_11, alpha=0.5, weights=(np.ones(t0)/t0), color='C7', ec='C7')
axs[1,1].axvline(0, c='k', linestyle=':')
xl = max(abs(bins_11))*1.1
axs[1,1].set_xlim(-xl, xl);